### Preparar ambiente

Primero descomentar las siguientes lineas e instalar PyAthena. Solo se hace la primera vez que se abre la instancia

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
!{sys.executable} -m pip install pytz

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### Conectar con pyathena

In [3]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

#### Lista de tablas de interes 

In [4]:
tabla_datos_input = f"veraset_02030409_2020" # Tabla original
tabla_datos_output = f"march_data_200_files" # Tabla modificada para continuar con la query

#### Eliminando tablas  

In [5]:
for tabla in [ tabla_datos_output ]:
        query = (f"DROP VIEW IF EXISTS {tabla} ")
        cursor.execute(query)
        print(f" Se ha eliminado la vista: {tabla} ")

 Se ha eliminado la vista: march_data_200_files 


#### Renombrar columnas de tabla original 

In [6]:
# partition_1 - month
# partition_2 - day
query = (f'''CREATE VIEW {tabla_datos_output}
AS SELECT  device_id AS caid,
        timestamp AS utc_timestamp,
        lat AS latitude,
        lon AS longitude,
        device_os AS id_type,
        accuracy AS accuracy,
        date AS date,
        split_part(date,'-',2) AS partition_1, 
        split_part(split_part(date,'-',3),' ',1) AS partition_2,
        country AS iso_country_code
FROM {tabla_datos_input} ''')
cursor.execute(query)
print(f" Se ha creado la vista: {tabla_datos_output} ")

 Se ha creado la vista: march_data_200_files 


In [14]:
query = f'''SELECT iso_country_code,
        COUNT(*) as n_pings,
        COUNT(distinct (caid, utc_timestamp)) as n_pings_unicos
FROM {tabla_datos_output}
WHERE partition_1 = '02' AND partition_2 = '01'
GROUP BY iso_country_code 
ORDER BY iso_country_code '''
cursor.execute(query).as_pandas()

,iso_country_code,n_pings,n_pings_unicos
0,AR,95647318,47823659
1,BO,7050922,3525461
2,BR,769432940,384716470
3,BZ,515158,257579
4,CL,39963640,19981820
5,CO,70212350,35106175
6,CR,5947550,2973775
7,CU,390768,195384
8,DO,10940040,5470020
9,EC,13491982,6745991
